<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-1-public/blob/main/C4/W3/ungraded_labs/C4_W3_Lab_2_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Using a multi-layer LSTM for forecasting

In this lab, you will use the same RNN architecure in the first lab but will instead stack [LSTM](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM) layers instead of `SimpleRNN`.

In [ ]:
try:
    %tensorflow_version 2.x
except:
    pass

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import to_categorical

print("\u2022 Using TensorFlow Version:", tf.__version__)

In [ ]:
!pip install -U -q PyDrive

In [ ]:
!pip install -q requests

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
import os
import json
import tempfile
import requests

In [ ]:
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update

In [ ]:
!apt-get install tensorflow-model-server

In [ ]:
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
fileDownloaded = drive.CreateFile({'id':'1lZeXYfOJnoY43bpX9ZMhDkG_xHzuMg33'})

In [ ]:
fileDownloaded.GetContentFile('dataset.csv')
df = pd.read_csv('dataset.csv', delimiter=',' )
df.head()

In [ ]:
X = df.iloc[:,0:2].values
y = df.iloc[:,2].values
labels = [1, 2, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
print(X[0:5])
print(y[0:5])
print(X.shape)
print(y.shape)

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder =  LabelEncoder()
y1 = encoder.fit_transform(y)
y1

In [ ]:
Y = pd.get_dummies(y1).values
print(Y[0:5])

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
print(X_train[:5])
print(y_train[:5])
print(X_train.shape)

In [ ]:
normalize = layers.Normalization()
normalize.adapt(X)

model = tf.keras.Sequential([
    normalize,
#    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(9, activation='softmax'),
  ])
model

In [ ]:
model.compile(optimizer='adam',
              loss='huber',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=50, epochs=500)

In [ ]:
y_pred = model.predict(X_test)
y_pred

In [ ]:
actual = np.argmax(y_test,axis=1)
predicted = np.argmax(y_pred,axis=1)
print(f"Actual: {actual}")
print(f"Predicted: {predicted}")

In [ ]:
MODEL_DIR = tempfile.gettempdir()

version = 1

export_path = os.path.join(MODEL_DIR, str(version))

if os.path.isdir(export_path):
    print('\nAlready saved a model, cleaning up\n')
    !rm -r {export_path}

tf.keras.models.save_model(
    model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

print('\nexport_path = {}'.format(export_path))
!ls -l {export_path}

In [ ]:
!saved_model_cli show --dir {export_path} --all

In [ ]:
import sys
# We need sudo prefix if not on a Google Colab.
if 'google.colab' not in sys.modules:
  SUDO_IF_NEEDED = 'sudo'
else:
  SUDO_IF_NEEDED = ''

!{SUDO_IF_NEEDED} apt update

In [ ]:
!{SUDO_IF_NEEDED} apt-get install tensorflow-model-server

In [ ]:
os.environ["MODEL_DIR"] = MODEL_DIR

In [ ]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=fashion_model \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1

In [ ]:
!tail server.log

In [ ]:
X_predict = np.array([[610, 5.9]])
data = json.dumps({"signature_name": "serving_default", "instances": X_predict.tolist()})
print(data)

In [ ]:
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/fashion_model:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)['predictions']

print('The condition is {}'.format(labels[np.argmax(predictions[0])]))